# Introduction to Data Science

# The CRISP-DM approach to Data Science Tasks

_________________________

# Business Understanding
                    
### The Business Understanding phase focuses on understanding the objectives and requirements of the project.  
+ Determine business objectives:  

O objetivo do nosso projeto é entender quais são os fatores que influenciam os jovens e a educação brasileira. Utilizando os dados do INEP (Instituto Nacional de Estudos e Pesquisas Educacionais), podemos coletar o município, uf, média de alunos por turma, a taxa de reprovação, a taxa de abandono, a situação socio-econômica e diversas outras informações sobre o Ensino Médio de milhares de escolas ao redor do país e usar isso pra nos ajudar a compreender o que está por trás de uma educação de sucesso (ou não).

+ Assess situation:  

A educação brasileira é extremamente heterogênea, contando desde escolas de elite, com atividades complementares e baixa quantidade de aluno por professor, até escolas públicas cujo corpo docente nem sempre possui a formação acadêmica necessária pra matéria que dá aula. Os dados para pesquisas desse nível são extremamente extensos e de fácil acesso, porém de fonte única (INEP), vez que são coletados pelo governo e disponibilizados para a população de forma gratuita e transparente.

+ Determine data mining goals: 

Coletar os dados sobre as escolas do ensino médio brasileiro, pra saber se elas são públicas ou privadas, urbanas ou rurais, medidas que comprovem  o engajamento do corpo discente com a educação (taxa de abandono, de reprovação) e a qualificação do corpo discente. Todos esses dados são disponibilizados pelo INEP e podem ser adquiridos utilizando a biblioteca basedosdados.

+ Produce project plan:  
    + Coletar os Indicadores Educacionais do INEP via biblioteca basedosdados.

    + Para fatores educativos, também realizar o web scraping desses dados.

    + Entender os dados, explorá-los e descrevê-los. 

    + Verificar quais são as tabelas e colunas relevantes para os objetivos e se desfazer do restante.

    + Filtrar os dados, selecionando apenas escolas do ensino médio.

    + Observar a correlação entre as variáveis e a taxa de abandono/taxa de reprovação.

    + Construir um modelo que preveja a taxa de abandono e reprovação de uma escola, baseando-se nos outros dados obtidos.

    + Separar os nossos dados em treino e teste, utilizando o treino para treinar o modelo e o teste para avaliar as previsões.

    + Fazer o relatório final e revisar o projeto.



In [1]:
import pandas as pd

# aquisição dos dados
import basedosdados as bd
import requests
from bs4 import BeautifulSoup as bs

# visualização
from matplotlib import pyplot as plt
import seaborn as sns

# Data Understanding  

+ Collect initial data:  
+ Describe data:  
+ Explore data:  
+ Verify data quality:  

## Collect initial data:  

In [2]:
# Define algumas variáveis úteis
dataset_id = "br_inep_indicadores_educacionais"
billing_project_id="trabalho-amd"

Download das tabelas do dataset:
(o código comentado baixa do site os .csv)

In [3]:
#bd.download(dataset_id=dataset_id, table_id='escola', savepath='data/escola.csv', billing_project_id=billing_project_id)
df_escola = pd.read_csv('data/escola.csv')

In [ ]:
# Estes datasets foram baixados porém não serão usados
# Estou deixando o código comentado pois podem vir a ser úteis depois
# A escolha de não usar os dados é porque a maior parte deles são os mesmos dados apenas agrupados por
'''
#bd.download(dataset_id=dataset_id, table_id='brasil', savepath='data/brasil.csv', billing_project_id=billing_project_id)
df_brasil = pd.read_csv('data/brasil.csv')

#bd.download(dataset_id=dataset_id, table_id='brasil_remuneracao_docentes', savepath='data/brasil_remuneracao_docentes.csv', billing_project_id=billing_project_id)
df_brasil_remuneracao = pd.read_csv('data/brasil_remuneracao_docentes.csv')

#bd.download(dataset_id=dataset_id, table_id='municipio', savepath='data/municipio.csv', billing_project_id=billing_project_id)
df_municipio = pd.read_csv('data/municipio.csv')

#bd.download(dataset_id=dataset_id, table_id='municipio_taxas_transicao', savepath='data/municipio_taxas_transicao.csv', billing_project_id=billing_project_id)
df_municipio_taxas = pd.read_csv('data/municipio_taxas_transicao.csv')

#bd.download(dataset_id=dataset_id, table_id='regiao', savepath='data/regiao.csv', billing_project_id=billing_project_id)
df_regiao = pd.read_csv('data/regiao.csv')

#bd.download(dataset_id=dataset_id, table_id='uf', savepath='data/uf.csv', billing_project_id=billing_project_id)
df_uf = pd.read_csv('data/uf.csv')

#bd.download(dataset_id=dataset_id, table_id='uf_remuneracao_docentes', savepath='data/uf_remuneracao_docentes.csv', billing_project_id=billing_project_id)
df_uf_remuneracao = pd.read_csv('data/uf_remuneracao_docentes.csv')

#bd.download(dataset_id=dataset_id, table_id='escola_nivel_socioeconomico', savepath='data/escola_nivel_socioeconomico.csv', billing_project_id=billing_project_id)
df_escola_nivel_socioeconomico = pd.read_csv('data/escola_nivel_socioeconomico.csv')
'''

Como pode ser justificado na sessão *2.b Describe Data*, o dataset `escola.csv` tem apenas a coluna `id_municipio`, porém nenhum identificador de qual município é aquele nem qual a sua UF. <br>
O IBGE classifica cada município com um código de 7 dígitos onde os dois primeiros representam a UF (referência: <https://www.ibge.gov.br/explica/codigos-dos-municipios.php>). <br>
Sendo assim, realizamos o scrapping dos códigos das UF presentes no link acima, mais para fins de aprendizado do que por questões práticas já que se tratavam de poucos dados

In [5]:
html = requests.get("https://www.ibge.gov.br/explica/codigos-dos-municipios.php")
soup = bs(html.text)
soup.prettify

<bound method Tag.prettify of 
<!DOCTYPE HTML>

<html lang="pt-BR">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="pt-br" http-equiv="Content-Language"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="Inflação é o nome dado ao aumento dos preços de bens e serviços. Ela é calculada pelos índices de preços, comumente chamados de índices de inflação." name="description"/>
<meta content="Instituto Brasileiro de Geografia e Estatística" name="author"/>
<meta content="Inflação, IPCA, Preços, INPC, Consumo, Compra" name="keywords"/>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-285486-1"></script>
<script>
        window.dataLayer = window.dataLayer || [];
        function gtag(){dataLayer.push(arguments);}
        gtag('js', new Date());

        gtag('config', 'UA-285486-1');
        gtag('config', 'UA-285486-76', {'groups':'ibgelab

In [6]:
# seleciona a tag table
table = soup.find('table')

# extrai o nome das colunas da tabela
column_names = []
for name in table.find('tr'):
    if name != "\n":
        column_names.append(name.text)

# extrai as linhas da tabela
trows = soup.find('tbody').find_all('tr')
tcols = []

# itera sobre as linhas
for tr in trows:
    # cria uma lista das colunas dessa linha
    col = tr.find_all('td')
    uf = col[0].text
    code = int(col[1].text.replace('ver municípios', ''))
    tcols.append([uf, code])

df_codigos_uf = pd.DataFrame(tcols, columns = column_names)
df_codigos_uf

,UFs,Códigos
0,Acre,12
1,Alagoas,27
2,Amapá,16
3,Amazonas,13
4,Bahia,29
5,Ceará,23
6,Distrito Federal,53
7,Espírito Santo,32
8,Goiás,52
9,Maranhão,21


## Describe Data

Começando pela tabela `escola` que é a mais importante do dataset, podemos ver que ela é formada por 208 colunas, destas 205 numéricas e 3 categóricas.

Algumas colunas notáveis são:
+ **rede:** 

municipal, estadual, federal ou privada;

+ **localização:**

urbana ou rural;

+ **atu**: Média de Alunos por Turma:

Quociente entre a matrícula total e o total de turmas da escola;

+ **had**: Média de Horas-Aula diária:

Percentual de alunos, em cada série, com idade superior à recomendada. Considerando 6 anos como a idade adequada para ingresso no fundamental e 15 para o ingresso no Ensino Médio;

+ **tdi**: Taxa de distorção idade-série:

Percentual de alunos, em cada série, com idade superior à recomendada. No Brasil, a idade recomendada para o ingresso no ensino médio ($1^o$ ano) é de 15 anos;

+ **taxa_aprovação/taxa_reprovação**:

Percentual de alunos aprovados/reprovados naquela determinada série;

+ **taxa_abandono**:

Percentual de alunos que abandonaram os estudos;

+ **dsu**: Percentual de Docentes com Curso Superior:

Indicador do nível de formação dos docentes em exercício;

+ **afd**: Adequação da Formação Docente:

Classificação do docente de acordo com a adequação de sua formação às disciplinas que leciona, ou seja, considera os professores que são formados em licenciatura na área da disciplina que leciona.

+ **ird_media_regularidade_docente**: Regularidade do corpo docente nas escolas de educação básica:

Avaliação da regularidade do corpo docente em um período de 5 anos. O IRD varia de 0 a 5, sendo que quanto mais ptóximo de 0, mais irregular é o docente, e quanto mais próximo de 5, mais regular ele é;

+ **id_municipio**:

Código do município no IBGE, o código é composto por 7 dígitos onde os dois primeiros indicam a Unidade Federativa a qual ele pertence;

+ **tnr**: 

Taxa de não resposta.

Referência para a explicação de cada índice:
<http://www.gestaouniversitaria.com.br/artigos/conheca-os-indicadores-que-detalham-as-tendencias-da-educacao-basica-brasileira>

In [7]:
df_escola.info()
df_escola.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637557 entries, 0 to 2637556
Columns: 208 entries, ano to icg_nivel_complexidade_gestao_escola
dtypes: float64(202), int64(3), object(3)
memory usage: 4.1+ GB


,ano,id_municipio,rede,id_escola,localizacao,atu_ei,atu_ei_creche,atu_ei_pre_escola,atu_ef,atu_ef_anos_iniciais,...,ied_ef_anos_finais_nivel_4,ied_ef_anos_finais_nivel_5,ied_ef_anos_finais_nivel_6,ied_em_nivel_1,ied_em_nivel_2,ied_em_nivel_3,ied_em_nivel_4,ied_em_nivel_5,ied_em_nivel_6,icg_nivel_complexidade_gestao_escola
0,2013,1100015,municipal,11024542,rural,NaN,NaN,NaN,5.0,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,nivel 1
1,2013,1100015,municipal,11025000,rural,NaN,NaN,NaN,6.0,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,nivel 1
2,2013,1100015,municipal,11025115,rural,11.0,NaN,11.0,13.0,NaN,...,30.8,7.7,7.7,NaN,NaN,NaN,NaN,NaN,NaN,nivel 2
3,2013,1100015,municipal,11047461,urbana,19.3,NaN,19.3,20.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nivel 2
4,2013,1100023,municipal,11007850,rural,NaN,NaN,NaN,18.1,17.2,...,71.4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,nivel 3
5,2013,1100023,estadual,11048620,urbana,NaN,NaN,NaN,29.9,27.8,...,33.3,44.4,5.6,0.0,0.0,0.0,46.6,46.7,6.7,nivel 6
6,2013,1100023,municipal,11007931,urbana,26.1,NaN,26.1,27.7,27.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nivel 5
7,2013,1100049,municipal,11026294,rural,13.0,NaN,13.0,17.1,17.6,...,42.9,0.0,14.3,NaN,NaN,NaN,NaN,NaN,NaN,nivel 2
8,2013,1100049,estadual,11026014,urbana,NaN,NaN,NaN,19.0,19.8,...,71.4,14.3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,nivel 3
9,2013,1100049,estadual,11026545,urbana,NaN,NaN,NaN,27.5,27.4,...,33.3,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,nivel 3


### Explore Data

# Data Preparation

+ Select data:   
+ Clean data:   
+ Construct data/transform data/feature engineering:   
+ Integrate data:   
+ (Re) Format data:   

A ideia é apenas trabalhar com os dados do ensino médio,portanto filtrando as colunas, temos:

In [84]:
# seleciona as colunas relacionadas ao EM:
em_cols = ['id_escola', 'ano', 'id_municipio', 'rede', 'localizacao']
for col in df_escola.columns:
    # se a coluna tem a string "em" seleciona ela
    if "em" in col:
        em_cols.append(col)

# redefine o dataset apenas com essas colunas
df_escola_em = df_escola[em_cols]

# remove as linhas que não tem dados do EM (consideramos que se não há taxa de aprovação, a escola não tem EM)
df_escola_em = df_escola_em.dropna(subset=['taxa_aprovacao_em'])

Também queremos adicionar as UFs de cada escola no dataset

In [86]:
# Código para adicionar o ID da UF
id_ufs = []

for index, row in df_escola_em.iterrows():
    id_uf = int(str(row['id_municipio'])[0:2])
    id_ufs.append(id_uf)

df_escola_em.insert(3, "id_uf", id_ufs, True)

In [60]:
# Código para adicionar o nome da UF no dataframe
"""
ufs = []

for index, row in df_escola_em.iterrows():
    id_uf = int(str(row['id_municipio'])[0:2])
    # ufs.append(df_codigos_uf.query(f"Códigos == {int(str(row['id_municipio'])[0:2])}")['UFs'])

ufs = pd.Series(ufs)

df_escola_em.insert(3, "uf", ufs, True)
"""

E aqui temos o dataset filtrado:

In [87]:
df_escola_em.info(show_counts=True)
df_escola_em.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350586 entries, 5 to 2637554
Data columns (total 59 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id_escola                       350586 non-null  int64  
 1   ano                             350586 non-null  int64  
 2   id_municipio                    350586 non-null  int64  
 3   id_uf                           350586 non-null  int64  
 4   rede                            350586 non-null  object 
 5   localizacao                     350586 non-null  object 
 6   atu_em                          350062 non-null  float64
 7   atu_em_1_ano                    342918 non-null  float64
 8   atu_em_2_ano                    336292 non-null  float64
 9   atu_em_3_ano                    329100 non-null  float64
 10  atu_em_4_ano                    16415 non-null   float64
 11  atu_em_nao_seriado              8344 non-null    float64
 12  had_em         

,id_escola,ano,id_municipio,id_uf,rede,localizacao,atu_em,atu_em_1_ano,atu_em_2_ano,atu_em_3_ano,...,afd_em_grupo_2,afd_em_grupo_3,afd_em_grupo_4,afd_em_grupo_5,ied_em_nivel_1,ied_em_nivel_2,ied_em_nivel_3,ied_em_nivel_4,ied_em_nivel_5,ied_em_nivel_6
5,11048620,2013,1100023,11,estadual,urbana,24.7,31.0,26.0,17.0,...,0.0,44.9,5.1,7.7,0.0,0.0,0.0,46.6,46.7,6.7
11,11027401,2013,1100049,11,estadual,urbana,22.2,25.5,19.5,21.0,...,0.0,45.3,7.8,0.0,0.0,0.0,0.0,100.0,0.0,0.0
19,11036125,2013,1100072,11,estadual,rural,18.3,16.0,16.0,23.0,...,0.0,52.8,0.0,8.3,0.0,0.0,0.0,85.7,14.3,0.0
36,11049014,2013,1100114,11,privada,rural,30.0,30.0,NaN,NaN,...,0.0,58.3,0.0,0.0,0.0,70.0,0.0,20.0,0.0,10.0
54,11017775,2013,1100155,11,estadual,urbana,27.4,30.1,29.0,22.0,...,1.8,27.0,2.3,0.0,0.0,3.5,14.3,53.6,14.3,14.3
73,11000937,2013,1100205,11,estadual,urbana,35.0,39.0,31.5,34.0,...,0.0,28.4,5.9,0.0,0.0,0.0,0.0,68.7,6.3,25.0
78,11038063,2013,1100205,11,estadual,urbana,32.8,32.8,29.0,44.0,...,5.0,32.7,7.8,0.0,0.0,0.0,27.8,44.4,16.7,11.1
95,11107804,2013,1100304,11,federal,urbana,23.7,27.8,23.0,18.8,...,13.8,15.8,6.1,0.0,0.0,12.5,65.0,17.5,5.0,0.0
97,11041951,2013,1100304,11,privada,urbana,22.7,19.0,18.0,31.0,...,7.7,43.6,2.5,0.0,0.0,0.0,0.0,61.5,7.7,30.8
120,11004428,2013,1100452,11,estadual,urbana,33.3,38.8,28.5,31.3,...,0.0,26.9,20.5,9.9,0.0,0.0,0.0,54.2,37.5,8.3


# Data Exploration

## IV. Modeling

+ Select modeling techniques: 
+ Generate test design: 
+ Build model: 
+ Assess model:  

## V. Evaluation                    

+ Evaluate results: 
+ Review process: 
+ Determine next steps: 

## VI. Deployment

+ Plan deployment: 
+ Plan monitoring and maintenance: 
+ Produce final report: 
+ Review project: 